# T5を転移学習（fine tuning）して応答生成モデルを作成する例
処理の流れ：ライブラリやデータの準備 → 学習の実行 → テストデータに対する応答生成

主な設定項目
* PRETRAINED_MODEL_NAME　使用する事前学習済みモデルの名前
* TRANSFER_SOURCE_MODEL　転移学習の起点となるモデルの名前（フォルダ名）
* TRANSFER_TARGET_MODEL　転移学習の結果得られるモデルを保存するフォルダ名
* INPUT_{DIR, FILENAME, FORMAT}　転移学習に使う学習データの設定
* OUTPUT_TEXT_{DIR, FILENAME}　生成された応答文の保存場所

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ライブラリやデータの準備

In [ ]:
# GPU情報の確認
!nvidia-smi

# 自分のGoogleドライブをマウントする
from google.colab import drive
drive.mount('/content/drive') # mount my google drive

# 作業用フォルダに移動する
%cd "/content/drive/MyDrive/Colab Notebooks/t5nlp"

Wed Jan 10 13:23:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0              29W /  70W |   5405MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# 使用する事前学習済みモデル
PRETRAINED_MODEL_NAME = "sonoisa/t5-base-japanese"

# 転移学習の起点となるモデル
TRANSFER_SOURCE_MODEL = PRETRAINED_MODEL_NAME  # 1段階目のfine tuningの場合
# TRANSFER_SOURCE_MODEL = "model_1"  # 2段階目のfine tuningの場合

# 転移学習の結果得られるモデル（事前に作業用フォルダの下にこの名前のフォルダを作っておく）
TRANSFER_TARGET_MODEL = "model_1"  # 1段階目のfine tuningで得られたモデルの保存場所の例
# TRANSFER_TARGET_MODEL = "model_2"  # 2段階目のfine tuningで得られたモデルの保存場所の例

# 転移学習に使う学習コーパスを置くフォルダ，ファイル名，ファイル形式
INPUT_DATA_DIR = "data1"
INPUT_DATA_FILENAME = "tweet_pairs_1m.txt"
INPUT_DATA_FORMAT = "tab"     # 発話文\t応答文\n
# INPUT_DATA_FORMAT = "newline" # 発話文\n応答文\n\n

# 生成された応答文を保存するフォルダ，ファイル名
OUTPUT_TEXT_DIR = "output"
OUTPUT_TEXT_FILENAME = "output.txt"

## 依存ライブラリのインストール

インストールでエラーが出る場合は，適宜バージョン指定などを調整する

In [ ]:
!pip install torch==2.1.0 torchtext==0.15.2 torchvision==0.15.2 torchaudio==2.1.0+cu121 pytorch-lightning

  Using cached torchtext-0.15.2-cp310-cp310-manylinux1_x86_64.whl (2.0 MB)
  Using cached torchvision-0.15.2-cp310-cp310-manylinux1_x86_64.whl (6.0 MB)
INFO: pip is looking at multiple versions of torchtext to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==2.1.0 and torchtext==0.15.2 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==2.1.0
    torchtext 0.15.2 depends on torch==2.0.1

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
!pip install -q transformers pytorch_lightning sentencepiece

## 文字列の正規化の定義

表記揺れを減らします。今回は[neologdの正規化処理](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)を一部改変したものを利用します。
処理の詳細はリンク先を参照してください。

In [ ]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

## 文字のノーマライズ化とデータ整形を実施

In [ ]:
import re
import pickle
from tqdm import tqdm

def normalize_text(text):
    # assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

all_data = []

with open(INPUT_DATA_DIR + '/' + INPUT_DATA_FILENAME, 'r', encoding='utf-8') as f:
    lines = f.readlines()
if INPUT_DATA_FORMAT == "tab" :
    for i in tqdm(range(0, len(lines) - 1)):
        text_str = normalize_text(lines[i].split('\t')[0])
        response_str = normalize_text(lines[i].split('\t')[1])
        if len(text_str) == 0 or len(response_str) == 0:
            print("empty str", i, lines[i])
        all_data.append({ "text": text_str, "response": response_str })
elif INPUT_DATA_FORMAT == "newline" :
    for i in tqdm(range(0, len(lines) - 1, 3)):
        text_str = normalize_text(lines[i])
        response_str = normalize_text(lines[i+1])
        if len(text_str) == 0: print("empty str", i, lines[i])
        if len(response_str) == 0: print("empty str", i+1, lines[i+1])
        all_data.append({ "text": text_str, "response": response_str })


100%|██████████| 9999/9999 [00:00<00:00, 12882.79it/s]


## データ分割

データセットを一定の比率でtrain/val/testに分割します（比率はコード中に指定）。

* trainデータ: 学習に利用するデータ
* valデータ: 学習中の精度評価等に利用するデータ
* testデータ: 学習結果のモデルの精度評価に利用するデータ

In [ ]:
import random
from tqdm import tqdm

random.seed(1234)
random.shuffle(all_data)

def to_line(data):
    text = data["text"]
    response = data["response"]

    assert len(text) > 0 and len(response) > 0
    return f"{text}\t{response}\n"

data_size = len(all_data)
train_ratio, val_ratio, test_ratio = 0.98, 0.01, 0.01

with open(INPUT_DATA_DIR + "/train.tsv", "w", encoding="utf-8") as f_train, \
    open(INPUT_DATA_DIR + "/val.tsv", "w", encoding="utf-8") as f_val, \
    open(INPUT_DATA_DIR + "/test.tsv", "w", encoding="utf-8") as f_test:

    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + val_ratio) * data_size:
            f_val.write(line)
        else:
            f_test.write(line)


9999it [00:00, 402468.67it/s]


作成されたデータを確認します。

形式: {テキスト}\t{レスポンス}

In [ ]:
!head -3 data1/test.tsv

今日もtantanに起こされました眠いです	自由気まま気の赴くままですものね
うんそうだねこんこんだね	どらえもん
お仕事お疲れ様です!	ありがとうです!


# 学習に必要なクラス等の定義

学習にはPyTorch/PyTorch-lightning/Transformersを利用します。

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir=INPUT_DATA_DIR,
    model_name_or_path=TRANSFER_SOURCE_MODEL,
    tokenizer_name_or_path=TRANSFER_SOURCE_MODEL,

    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)

## TSVデータセットクラス

TSV形式のファイルをデータセットとして読み込みます。  
形式は"{text}\t{response}"です。

In [ ]:
from tqdm import tqdm
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)

        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask,
                "target_ids": target_ids, "target_mask": target_mask}

    def _make_record(self, text, response):
        # 応答生成タスク用の入出力形式に変換する。
        input = f"{text}"
        target = f"{response}"
        return input, target

    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for i, line in tqdm(enumerate(f)):
                line = line.strip().split("\t")
                assert len(line) == 2
                assert len(line[0]) > 0
                assert len(line[1]) > 0

                text = line[0]
                response = line[1]

                input, target = self._make_record(text, response)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True,
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True,
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)

                if (i >= 1000000): # 対話データを100万までで制限する
                    break


## 学習処理クラス

[PyTorch-Lightning](https://github.com/PyTorchLightning/pytorch-lightning)を使って学習します。

PyTorch-Lightningとは、機械学習の典型的な処理を簡潔に書くことができるフレームワークです。

対話データセットが大きいため、かなりかかります

（input_size=24,output_size=24）

（train_dataが大きすぎるため100万に制限）

In [ ]:
import pickle
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        #rewrite
        #self.hparams = hparams
        self.save_hyperparameters(hparams)

        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None,
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked),
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters()
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters()
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=self.hparams.learning_rate,
                          eps=self.hparams.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams.warmup_steps,
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer,
            data_dir=args.data_dir,
            type_path=type_path,
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)

    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, type_path="val.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset,
                          batch_size=self.hparams.train_batch_size,
                          drop_last=True, shuffle=True, num_workers=0)#4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset,
                          batch_size=self.hparams.eval_batch_size,
                          num_workers=0)#4)


# 転移学習を実行

In [ ]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  24,  # 入力文の最大トークン数
    "max_target_length": 24,  # 出力文の最大トークン数
    "train_batch_size":  8,  # バッチサイズ（train）
    "eval_batch_size":   8,  # バッチサイズ（eval）
    "num_train_epochs":  10, # トレーニングのエポック数
    })
args = argparse.Namespace(**args_dict)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    #gpus=args.n_gpu,
    accelerator="auto",
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    #amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
)

In [ ]:
# 転移学習の実行（GPUを利用して1エポック6時間程度）
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
model.tokenizer.save_pretrained(TRANSFER_TARGET_MODEL)
model.model.save_pretrained(TRANSFER_TARGET_MODEL)

del model

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs

0it [00:00, ?it/s]
291it [00:00, 2904.29it/s]
582it [00:00, 2830.75it/s]
904it [00:00, 3001.72it/s]
1225it [00:00, 3081.07it/s]
1551it [00:00, 3143.47it/s]
1866it [00:00, 3085.12it/s]
2175it [00:00, 3059.13it/s]
2486it [00:00, 3072.68it/s]
2794it [00:00, 2906.84it/s]
3087it [00:01, 2907.56it/s]
3379it [00:01, 2865.34it/s]
3706it [00:01, 2982.50it/s]
4029it [00:01, 3054.93it/s]
4336it [00:01, 3000.86it/s]
4674it [00:01, 3111.86it/s]
4994it [00:01, 3137.66it/s]
5336it [00:01, 3219.50it/s]
5659it [00:01, 3101.76it/s]
5971it [00:01, 3043.63it/s]
6300it [00:02, 3113.91it/s]
6613it [00:02, 3007.85it/s]
6918it [00:02, 3017.42it/s]
7258it [00:02, 3128.13it/s]
7598it [00:02, 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


# 学習済みモデルの読み込み

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(TRANSFER_TARGET_MODEL, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(TRANSFER_TARGET_MODEL)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# テストデータに対する応答生成

In [ ]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
#test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv",
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "sample_test_data2.tsv",
                          input_max_len=args.max_input_length,
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=8, num_workers=0)

trained_model.eval()

inputs = []
outputs = []
targets = []

for i, batch in tqdm(enumerate(test_loader)):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    output = trained_model.generate(input_ids=input_ids,
        attention_mask=input_mask,
        max_length=args.max_target_length,
        temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
        repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        )

    output_text = [tokenizer.decode(ids, skip_special_tokens=True,
                            clean_up_tokenization_spaces=False)
                for ids in output]
    target_text = [tokenizer.decode(ids, skip_special_tokens=True,
                               clean_up_tokenization_spaces=False)
                for ids in batch["target_ids"]]
    input_text = [tokenizer.decode(ids, skip_special_tokens=True,
                               clean_up_tokenization_spaces=False)
                for ids in input_ids]

    inputs.extend(input_text)
    outputs.extend(output_text)
    targets.extend(target_text)

    if i > 1000: break


0it [00:00, ?it/s]

0it [00:00, ?it/s]

## 生成結果確認

形式
- generated: 生成された応答
- actual: 人が作成した応答（正解）
- text: 応答元の文章


In [ ]:
with open(OUTPUT_TEXT_DIR + '/' + OUTPUT_TEXT_FILENAME, "w", encoding="utf-8") as f_out:
    for output, target, input in zip(outputs, targets, inputs):
        f_out.write(input + "\n")
        f_out.write(output + "\n\n")

In [ ]:
for output, target, input in zip(outputs, targets, inputs):
    print("text:      " + input)
    print("generated: " + output)
    print("actual:    " + target)
    print()

text:      起きてたんかい!
generated: おはようございます
actual:    当たり前じゃん

text:      眠くないんだよね
generated: 眠いんだよなぁ
actual:    何でそんなに眠くならんのやぁ

text:      ありがとう!
generated: やったぁ!
actual:    はい

text:      ずっとおじさんの掌の上で転がされ続けるんだと思います
generated: 掌の上で転がされ続けるの可哀想
actual:    それを喜んで転がりにいく私たち

text:      可愛いからええぞ
generated: かわいいからええぞ
actual:    はい

text:      この映画面白いよね
generated: めちゃくちゃ面白かった!
actual:    はい

text:      失敗しちゃった
generated: めちゃくちゃ楽しかった!これからも続けていけると信じて頑張る
actual:    はい

text:      プログラミングは得意ですか?
generated: 得意です!
actual:    はい

text:      質問を答えて
generated: おれはこう答えます
actual:    はい

text:      今日はどんな気分?
generated: リラックスタイム!
actual:    はい



# 任意の文章に対する応答生成

応答を10個、自動生成してみます。

以下のコードでは多様性を生むために色々generateメソッドのパラメータを設定しています。パラメータの詳細は下記リンク先を参照してください。

- [generateメソッドのパラメータの意味](https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate)

In [ ]:
body = """
今日はどうですか？
"""

In [ ]:
MAX_SOURCE_LENGTH = args.max_input_length   # 入力される記事本文の最大トークン数
MAX_TARGET_LENGTH = args.max_target_length  # 生成されるタイトルの最大トークン数

def preprocess_body(text):
    return normalize_text(text.replace("\n", " "))

# 推論モード設定
trained_model.eval()

# 前処理とトークナイズを行う
inputs = [preprocess_body(body)]
batch = tokenizer.batch_encode_plus(
    inputs, max_length=MAX_SOURCE_LENGTH, truncation=True,
    padding="longest", return_tensors="pt")

input_ids = batch['input_ids']
input_mask = batch['attention_mask']
if USE_GPU:
    input_ids = input_ids.cuda()
    input_mask = input_mask.cuda()

# 生成処理を行う
outputs = trained_model.generate(
    input_ids=input_ids, attention_mask=input_mask,
    max_length=MAX_TARGET_LENGTH,
    temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
    num_beams=10,             # ビームサーチの探索幅
    diversity_penalty=1.0,    # 生成結果の多様性を生み出すためのペナルティ
    num_beam_groups=10,       # ビームサーチのグループ数
    num_return_sequences=10,  # 生成する文の数
    repetition_penalty=1.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
)

# 生成されたトークン列を文字列に変換する
generated_titles = [tokenizer.decode(ids, skip_special_tokens=True,
                                     clean_up_tokenization_spaces=False)
                    for ids in outputs]

# 生成されたタイトルを表示する
for i, title in enumerate(generated_titles):
    print(f"{i+1:2}. {title}")

 1. 昨日と変わらず、お客さんは多くて賑わいました。
 2. どういたしまして今日はいかがでしたか?
 3. 明日はどうですか?
 4. 今から帰ります!
 5. 今日は、どうでしたか?
 6. それでいいんですか?
 7. 明日もよろしくです!
 8. 大丈夫ですよ!
 9. それは、おれです今日は、どうですか?
10. 昨日と変わらずに、今日もよろしくお願いします!
